In [2]:
from langchain_ollama import OllamaLLM
from langchain.prompts import MessagesPlaceholder, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from IPython.display import Markdown, display
import logging
import os
import requests

In [3]:
# =========================
# General Config
# =========================
# Set up for bypassing system proxy
os.environ["no_proxy"] = "127.0.0.1,localhost"

# Set up logging for better error handling
DISABLE_LOGGING = True  # Set to True to disable logging, False to enable
logging.basicConfig(
    level=logging.CRITICAL if DISABLE_LOGGING else logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)

# ================================
# Configure Ollama host and port
# ================================
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "127.0.0.1")
OLLAMA_PORT = os.getenv("OLLAMA_PORT", "11434")
OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

# ================================
# List of models to iterate over
# ================================
models = [
    {"name": "llama3.2:3b", "description": "Llama 3.2"},
    {"name": "gemma3:latest", "description": "Gemma 3 (4.3B)"},
    {"name": "deepseek-r1:1.5b", "description": "DeepSeek-R1 (1.8B)"}
]

# =========================
# Test server connection
# =========================
def test_server_connection(url: str) -> bool:
    try:
        response = requests.get(url, timeout=5)
        logger.info(f"Ollama server status: {response.status_code}")
        return True
    except requests.RequestException as e:
        logger.error(f"❌ Failed to connect to Ollama server at {url}: {e}")
        return False

In [4]:
# ===========================
# Define the prompt template
# ===========================

# version 1: without any memmory
# prompt = ChatPromptTemplate(
#     input_variables=["content"],
#     messages=[
#         HumanMessagePromptTemplate.from_template("{content}")
#     ]
# )

# version 2: with memmory
# memory = ConversationBufferMemory(memory_key="message_history")
# prompt = ChatPromptTemplate(
#     input_variables=["input", "messages"],
#     messages=[
#         MessagesPlaceholder(variable_name="message_history"),
#         HumanMessagePromptTemplate.from_template("{input}")
#     ]
# )

memory = ConversationBufferMemory(memory_key="history")
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

# =========================
# Input parameters
# =========================
# input_params = {"language": "python", "task": "return a list of 5 integers"}


C:\Users\Hossein\AppData\Local\Temp\ipykernel_788\2121765332.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history")


In [ ]:
input_msg = input(">> ")
inputs = {"input": input_msg, "history": memory.load_memory_variables({})["message_history"]}
llm = OllamaLLM(model=models[0], base_url=OLLAMA_URL, timeout=30)
chain = prompt | llm
result = chain.invoke(params)

In [10]:
# =================================================
# Function to invoke a model and display the result
# =================================================
def run_model(model: dict, prompt: ChatPromptTemplate, params: dict) -> None:
    try:
        logger.info(f"🚀 Running model: {model['description']} ({model['name']})")

        llm = OllamaLLM(model=model["name"], base_url=OLLAMA_URL, timeout=30)
        chain = prompt | llm
        result = chain.invoke(params)

        return result
        # display(Markdown(f"### {model['description']}\n```{params['language']}\n{result}\n```"))
    except Exception as e:
        logger.error(f"⚠️ Error with {model['description']}: {e}")

In [ ]:
if not test_server_connection(OLLAMA_URL):
    exit(1)

#Run One Model
while True:
    input_msg = input(">> ")
    inputs = {"input": input_msg, "history": memory.load_memory_variables({})["message_history"]}
    print( run_model(models[0], prompt, {"input":inputs}) )

# # Run each model
# for model in models:
#     run_model(model["name"], model["description"], code_prompt, input_params)

None
None
None


In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_ollama import OllamaLLM
import os

# Setup memory
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# Setup prompt with a placeholder for the chat history
prompt = ChatPromptTemplate(
    input_variables=["content", "history"],
    messages=[
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# Setup LLM
models = [
    {"name": "llama3.2:3b", "description": "Llama 3.2"},
    {"name": "gemma3:latest", "description": "Gemma 3 (4.3B)"},
    {"name": "deepseek-r1:1.5b", "description": "DeepSeek-R1 (1.8B)"}
]
# ================================
# Configure Ollama host and port
# ================================
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "127.0.0.1")
OLLAMA_PORT = os.getenv("OLLAMA_PORT", "11434")
OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

chat = OllamaLLM(model=models[0]["name"], base_url=OLLAMA_URL, timeout=30)

chain = prompt | chat

while True:
    content = input(">> ")

    # Load chat history from memory
    history = memory.load_memory_variables({})["history"]

    # Run the chain with input content and history
    result = chain.invoke({"content": content, "history": history})

    # Print AI response
    print(result)

    # Save conversation context (user input and AI response)
    memory.save_context({"content": content}, {"response": result})

RemoteProtocolError: Server disconnected without sending a response.